In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.distributions.kl import kl_divergence
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D

In [17]:
def f(point, alpha):
    return point.sum(dim=1).mul(alpha)

num_datapoints = 2000
alpha_range = [-2, 2]
x_dimension = 2
y_dimension = 1

a = torch.linspace(-2, 2, 100)
b = torch.linspace(-2, 2, 100)
x_t = a.repeat(100).unsqueeze(0).transpose(0,1)
y_t = b.repeat(100,1).t().contiguous().view(-1).unsqueeze(0).transpose(0,1)

mesh = torch.cat([x_t, y_t], dim=1)
datapoints = []
for _ in range(num_datapoints):
    num = np.random.uniform(alpha_range[0], alpha_range[1])
    value = f(mesh, num).unsqueeze(0).transpose(0,1)
    datapoints.append([mesh, value])

points = np.random.choice(100**2, 50)
testpoints = [mesh[points], f(mesh[points], 1).unsqueeze(0).transpose(0,1)]

In [18]:
class encoder(nn.Module):
    def __init__(self, output_sizes):
        super(encoder, self).__init__()
        self.output_sizes = output_sizes
        
        self.fc1 = nn.Linear(x_dimension+y_dimension, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, self.output_sizes)
        self.relu = nn.ReLU()
        
    def forward(self, x, y):
        #x and y are 1xn dimensional torch tensors
        
        xy = torch.cat([x, y], dim=1)
        l1 = self.relu(self.fc1(xy))
        l2 = self.relu(self.fc2(l1))
        R = self.fc3(l2)
    
        r_agg = torch.mean(R, dim=0).reshape(1, self.output_sizes)
        return r_agg

In [19]:
class r_to_z(nn.Module):
    def __init__(self, encoded_size):
        super(r_to_z, self).__init__()
        self.encoded_size = encoded_size
        
        self.fc1 = nn.Linear(self.encoded_size, 10)
        self.fc2_mu = nn.Linear(10, self.encoded_size)
        self.fc2_logvar = nn.Linear(10, encoded_size)
        self.relu = nn.ReLU()
        
    def forward(self, r):
        mu = self.fc2_mu(self.relu(self.fc1(r))).reshape(3)
        logvar = self.fc2_logvar(self.relu(self.fc1(r)))
        std = torch.exp(logvar.mul(1/2)).reshape(3)
        dist = torch.distributions.Normal(mu, std)
        
        mu_out = mu.reshape(self.encoded_size, 1)
        sigma_out = std.reshape(self.encoded_size, 1)
        return mu_out, sigma_out, dist

In [20]:
class decoder(nn.Module):
    def __init__(self, encoded_size):
        super(decoder, self).__init__()
        self.encoded_size = encoded_size
        
        self.fc1 = nn.Linear(self.encoded_size + x_dimension, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc21 = nn.Linear(10, 10)
        self.fc3_mu = nn.Linear(10, y_dimension)
        self.fc3_sigma = nn.Linear(10, y_dimension)
        self.relu = nn.ReLU()
        
    def forward(self, z, x):
        #x and z are 1xn dimensional torch tensors
        
        zmulti = torch.cat([z for i in range(x.size()[0])], dim=1).transpose(0,1)
        xz = torch.cat([x, zmulti], dim=1)
        
        l1 = self.relu(self.fc1(xz))
        l2 = self.relu(self.fc2(l1))
        l21 = self.relu(self.fc21(l2))
        out_mu = self.fc3_mu(l21)
        out_logvar = self.fc3_sigma(l21)
        out_sigma = torch.exp(out_logvar.mul(1/2))
        
        dist = torch.distributions.Normal(out_mu, out_sigma)
        
        return out_mu, out_sigma, dist

In [21]:
class CNP(nn.Module):
    def __init__(self, encoded_size):
        super(CNP, self).__init__()
        self.encoded_size = encoded_size
        self._encoder = encoder(encoded_size)
        self._rz = r_to_z(encoded_size)
        self._decoder = decoder(encoded_size)
        
    def forward(self, context_x, context_y, target_x, target_y=None):
        en_mu, en_sigma, en_dist = self._rz(self._encoder(context_x, context_y))
        if target_y is not None:
            t_en_mu, t_en_sigma, t_en_dist = self._rz(self._encoder(target_x, target_y))
            representation = t_en_dist.rsample().unsqueeze(0).transpose(0,1)
        else:
            representation = en_dist.rsample().unsqueeze(0).transpose(0, 1)
            t_en_dist = None

        mu, sigma, dist = self._decoder(representation, target_x)
        
        if target_y is not None:
            log_p = dist.log_prob(target_y)
            log_p = log_p.sum()
            
            MSE = (mu - target_y.transpose(0,1)).sum()
        else:
            log_p = None
            MSE = None
        return mu, sigma, log_p, en_dist, t_en_dist

In [22]:
cnp = CNP(3)
optimizer = torch.optim.Adam(cnp.parameters(), lr=3e-3)
num_test_maximum = 0.8*100**2
plot_frequency = 1
alpha = torch.tensor(1)

def compute_kernel(x, y):
    return (-(x - y).pow(2)).exp()

def MMD(z_prior, z_posterior, num_samples):
    target_samples_1 = z_prior.sample((num_samples,))
    context_samples_1 = z_posterior.sample((num_samples,))
    target_samples_2 = z_prior.sample((num_samples,))
    context_samples_2 = z_posterior.sample((num_samples,))
    
    t_kernel = compute_kernel(target_samples_1, target_samples_2)
    c_kernel = compute_kernel(context_samples_1, context_samples_2)
    ct_kernel = compute_kernel(target_samples_1, context_samples_1) + compute_kernel(target_samples_2, context_samples_2)
    return (t_kernel + c_kernel - ct_kernel).mean()
    

def lossf(log_p, z_prior, context_z_posterior, test_z_posterior, alpha):
    KL =  MMD(test_z_posterior, context_z_posterior, 10).prod()
    return -log_p + alpha*KL


def train(data, cnp, epochs, test_data):
    cnp.train()
    for epoch in range(epochs):
        total_loss = 0
        iteration = 0
        for function in data:
            optimizer.zero_grad()
            num_points = function[0].size()[0]
            perm = torch.randperm(num_points)
            num_context = np.random.randint(num_points - num_test_maximum, num_points)
            context_x = function[0][perm][0:num_context]
            context_y = function[1][perm][0:num_context]
            test_x = function[0][perm][num_context:num_points]
            test_y = function[1][perm][num_context:num_points]
            mu, sigma, log_p, en_dist, t_en_dist = cnp(context_x, context_y, test_x, test_y)
            loss = lossf(log_p, torch.distributions.Normal(torch.zeros(cnp.encoded_size), 1), en_dist, t_en_dist, alpha)
            loss.backward()
            optimizer.step()
            
            total_loss += loss
            iteration += 1
            
            if iteration % 200 == 0:
                print(iteration)
        
        if epoch % plot_frequency == 0:
            print("EPOCH: {}, LOSS {}".format(epoch, total_loss))
            
            fig = pyplot.figure()
            ax = Axes3D(fig)
            
            test_x = test_data[0]
            test_y = test_data[1]
            
            linspace = mesh
            
            mu, sigma, _, _, _ = cnp(test_x, test_y, linspace)
            lin = (linspace.numpy()[0])
            low = np.array((mu-sigma).detach().numpy().T[0])
            high = np.array((mu+sigma).detach().numpy().T[0])
            
            ax.scatter(linspace.numpy()[:,0], linspace.numpy()[:,1], mu.detach().numpy(), c='blue', alpha=0.05)
            ax.scatter(test_x.numpy()[:,0], test_x.numpy()[:,1], test_y.numpy(), c='black', alpha=0.5)
            plt.show()

            
def plot_priors(np, number):
    norm = torch.distributions.Normal(torch.zeros(np.encoded_size), 1)
    for i in range(number):
        z = norm.rsample().unsqueeze(0).transpose(0, 1)
        lin = torch.linspace(-2, 2, 100).unsqueeze(0)
        ys, _, _ = np._decoder(z, lin)
        yplot = ys.transpose(0,1).squeeze(0).detach().numpy()
        xplot = lin.squeeze(0).numpy()
        plt.plot(xplot, yplot)

In [ ]:
train(datapoints, cnp, 100, testpoints)